In [2]:
from datetime import datetime
from plyer import notification

def alerta(nivel, base, etapa):
    '''
        Alerta de falha de carregamento de base de dados
    '''
    now = str(datetime.now())

    msg = f"Falha no carregamento da base {base} na etapa {etapa}.\n{now}"


    if nivel == 1:
        title = 'ATENÇÃO: Alerta Baixo'
    elif nivel == 2:
        title = 'ATENÇÃO: Alerta Médio'
    elif nivel  == 3:
        title = 'ATENÇÃO: Alerta Alto'
    else:
         alerta(nivel = 2,
            base = f" " + msg,
            etapa = f"Falha na requisição ")

    notification.notify(
            title=title,
            message=msg,
            app_name='alerta',
            timeout=10
        )
    return



In [26]:
import pandas as pd
import requests

import sqlite3

conn = sqlite3.connect('banco_teste.db')
cursor = conn.cursor()

url1 = "https://brasilapi.com.br/api/cptec/v1/cidade"
url2 = "https://brasilapi.com.br/api/cptec/v1/clima/capital" #não utilizado na atividade
url3 = "https://brasilapi.com.br/api/cptec/v1/clima/previsao/{cityCode}" #não utilizado na atividade

# Função para permitir a extração dos dados e geração da tabela (será incrementado posteriormente o retorno para permitir as manipulações)
def carregar_dados(url):
    response = requests.get(url)
    if response.status_code == 200:
        data = response.json()
        df = pd.DataFrame(data)
        return df
    else:
        data = []
        
        alerta(nivel = 2,
            base = f"API " + url,
            etapa = f"Falha na requisição. Código de status: " + str(response.status_code))
        return df
    

def salvar_banco(dados):
    dados.to_sql('tabela_estados', conn, if_exists='replace', index=False)
        

estados = carregar_dados(url1)
estados
estados2 = carregar_dados(url2)
estados2
try:
    estados3 = carregar_dados(url3)
    estados3
except:
    print("Erro ao carregar")


df = estados.rename(columns={'nome': 'Nome', 'id': 'ID', 'estado': 'Estado'})
df['ID'] = df['ID'].astype(str)
df['Estado'].fillna('n/d', inplace=True)
grouped_df = df.groupby('Estado')['ID'].count().reset_index()


salvar_banco(grouped_df)
grouped_df




Erro ao carregar


,Estado,ID
0,AL,1
1,BA,5
2,CE,1
3,ES,1
4,GO,1
5,MA,6
6,MG,6
7,MT,1
8,PA,4
9,PB,5
